<a href="https://colab.research.google.com/github/lhb0803/helmet-project/blob/main/colab/construction_image_detection_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt

Cloning into 'yolov5'...
remote: Enumerating objects: 10200, done.
remote: Total 10200 (delta 0), reused 0 (delta 0), pack-reused 10200
Receiving objects: 100% (10200/10200), 10.42 MiB | 30.74 MiB/s, done.
Resolving deltas: 100% (7073/7073), done.
/content/yolov5
     |████████████████████████████████| 596 kB 5.0 MB/s 


In [2]:
import torch
from IPython.display import Image, clear_output

clear_output()
print(f"Setup complete. Using torch {torch.__version__} \
    ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.10.0+cu111     (Tesla P100-PCIE-16GB)


# Get Ready Data

* [ACID](https://www.acidb.ca/dataset) : advanced (6000 images) 
* [MPHB](http://parnec.nuaa.edu.cn/_upload/tpl/02/db/731/template731/pages/xtan/MPHB.html) : human pose data

In [3]:
import os
from pathlib import Path
from xml.dom.minidom import parse
from shutil import copyfile
from PIL import Image
from tqdm import tqdm

from google.colab import drive

import numpy as np
import pandas as pd

drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%ls

CONTRIBUTING.md  Dockerfile  LICENSE    requirements.txt*  tutorial.ipynb
data/            export.py   models/    setup.cfg          utils/
detect.py        hubconf.py  README.md  train.py           val.py


## Download ACID Data

In [5]:
# Download from my google drive
%cp -R ../drive/MyDrive/Helmet-Project/ACID_6000.zip data/ACID.zip
!unzip -q data/ACID.zip -d data/
%mv ./data/ACID_6000/JPEGImages/ ./data/ACID_6000/Images/ 
%rm -f ./data/ACID_6000/Images/.DS_Store
%rm -f ./data/ACID_6000/Annotations/.DS_Store

## Download MPHB data

In [6]:
# Download from my google drive
%cp -R ../drive/MyDrive/Helmet-Project/Human-Body.zip data/Human.zip
!unzip -q data/Human.zip -d data/
%mkdir ./data/Human
%mv ./data/Human\ Body\ Image/ ./data/Human/Images/
%mkdir ./data/Human/Annotations

## Preprocess data for YOLOv5 format

In [ ]:
classes = [
           'person', 
           'excavator', 'compactor', 'dozer', 'grader', 'dump_truck', 
           'cement_truck', 'wheel_loader', 'backhoe_loader',
           'tower_crane', 'mobile_crane'
        ]

In [ ]:
ACID_IMG_DIR = 'data/ACID_6000/Images/'
ACID_ANNOT_DIR = 'data/ACID_6000/Annotations/'

HUMAN_IMG_DIR = 'data/Human/Images/'
HUMAN_ANNOT_DIR = 'data/Human/Annotations/'

IMG_DIR = 'data/construction/images/'
LABEL_DIR = 'data/construction/labels/'

In [ ]:
!mkdir data/construction
!mkdir data/construction/labels
!mkdir data/construction/images

### ACID Preproccessing

In [ ]:
xml_files = os.listdir(ACID_ANNOT_DIR)

for file_name in tqdm(xml_files):
    file_path = ACID_ANNOT_DIR + file_name
    dom = parse(file_path)
    root = dom.documentElement

    img_size = root.getElementsByTagName("size")[0]
    img_w = int(img_size.getElementsByTagName("width")[0].childNodes[0].data)
    img_h = int(img_size.getElementsByTagName("height")[0].childNodes[0].data)

    objects = root.getElementsByTagName("object")

    bbox_list = []
    for box in objects:
        cls_name = box.getElementsByTagName("name")[0].childNodes[0].data
        xmin = int(box.getElementsByTagName("xmin")[0].childNodes[0].data)
        ymin = int(box.getElementsByTagName("ymin")[0].childNodes[0].data)
        xmax = int(box.getElementsByTagName("xmax")[0].childNodes[0].data)
        ymax = int(box.getElementsByTagName("ymax")[0].childNodes[0].data)

        bbox_list.append([cls_name, xmin, ymin, xmax, ymax])
    
    save_path = LABEL_DIR + file_name.split('.')[0] + '.txt'
    with open(save_path, 'w') as f:
        annotation = ""
        for bbox in bbox_list:
            cls_name, xmin, ymin, xmax, ymax = bbox
            cls_num = classes.index(cls_name)

            x = (xmin + xmax) / 2 / img_w
            y = (ymin + ymax) / 2 / img_h
            w = (xmax - xmin) / img_w
            h = (ymax - ymin) / img_h

            annotation += f"{cls_num} {x} {y} {w} {h}\n"
        f.write(annotation)

100%|██████████| 6000/6000 [00:02<00:00, 2075.50it/s]


### MPHB Preprocessing

In [ ]:
file_box_dict = {}

with open('./data/MPHB-label-txt/MPHB-label.txt', 'r') as f:
    while True:
        l = f.readline()
        if not l:
            break
        
        l_splitted = l.split()
        if len(l_splitted) == 2: # idx: 00001
            f_name = l_splitted[1]
            bbox_list = []
        elif len(l_splitted) == 4: # bbox
            bbox_list.append([float(v) for v in l_splitted])
        elif len(l_splitted) == 3: # source: MPII: 000001163.jpg
            f_extension = l_splitted[-1].split('.')[1]
            file_box_dict[f_name + '.' +f_extension] = bbox_list

print(len(file_box_dict))

26675


In [ ]:
for file_name, bbox_list in file_box_dict.items():
    img_path = HUMAN_IMG_DIR + file_name

    img_w, img_h = Image.open(img_path).size

    save_path = LABEL_DIR + file_name.split('.')[0] + '.txt'
    with open(save_path, 'w') as f:
        annotation = ""
        for bbox in bbox_list:
            cls_num = 0
            left, top, right, bottom = bbox
            x = (left + right) / 2 / img_w
            y = (top + bottom) / 2 / img_h
            w = (right - left) / img_w
            h = (bottom - top) / img_h
            annotation += f"{cls_num} {x} {y} {w} {h}\n"
        f.write(annotation)

## Train - Valid Split

In [ ]:
from sklearn.model_selection import train_test_split
import random
random.seed(76)

# ACID
ACID_image_list = os.listdir(ACID_IMG_DIR)
ACID_train_list, ACID_valid_list = train_test_split(ACID_image_list, test_size=0.2, random_state=76)

# HUMAN
num_to_select = 1000
HUMAN_image_list = random.sample(os.listdir(HUMAN_IMG_DIR), num_to_select)
HUMAN_train_list, HUMAN_valid_list = train_test_split(HUMAN_image_list, test_size=0.2, random_state=76)

print('total :',len(ACID_image_list) + len(HUMAN_image_list))
print('train :',len(ACID_train_list) + len(HUMAN_train_list))
print('val   :',len(ACID_valid_list) + len(HUMAN_valid_list))

total : 7000
train : 5600
val   : 1400


In [ ]:
def copy_data(file_list, img_labels_root, imgs_source, mode):

    root_file = Path(IMG_DIR + mode)
    if not root_file.exists():
        print(f"Path {root_file} does not exist")
        os.makedirs(root_file)

    root_file = Path(LABEL_DIR + mode)
    if not root_file.exists():
        print(f"Path {root_file} does not exist")
        os.makedirs(root_file)

    for file in tqdm(file_list):
        img_name, img_format = file.split('.')
        img_src_file = imgs_source  + img_name + f".{img_format}"
        label_src_file = img_labels_root + img_name + '.txt'

        # Copy image
        img_dict_file = IMG_DIR + mode + '/' + img_name + f".{img_format}"
        copyfile(img_src_file, img_dict_file)
        # Delete image
        os.remove(img_src_file)

        # Copy label
        label_dict_file = LABEL_DIR + mode + '/' + img_name + '.txt'
        copyfile(label_src_file, label_dict_file)
        # Delete label
        os.remove(label_src_file)


In [ ]:
# ACID
copy_data(ACID_train_list, LABEL_DIR, ACID_IMG_DIR, "train")
copy_data(ACID_valid_list, LABEL_DIR, ACID_IMG_DIR, "val")

# HUMAN
copy_data(HUMAN_train_list, LABEL_DIR, HUMAN_IMG_DIR, "train")
copy_data(HUMAN_valid_list, LABEL_DIR, HUMAN_IMG_DIR, "val")

Path data/construction/images/train does not exist
Path data/construction/labels/train does not exist


100%|██████████| 4800/4800 [00:08<00:00, 581.05it/s]


Path data/construction/images/val does not exist
Path data/construction/labels/val does not exist


100%|██████████| 200/200 [00:00<00:00, 3348.61it/s]


## Create YAML file for dataset configuration

In [ ]:
import yaml

data_config_dict = {
    'train': IMG_DIR + 'train',
    'val': IMG_DIR + 'val',
    'nc': f'{len(classes)}',
    'names': classes
}

with open('data/construction.yaml', 'w+') as f:
    documents = yaml.dump(data_config_dict, f)

# Train

## Set visualization setting with W&B tool

In [ ]:
# Weights & Biases  (optional)
%pip install -q wandb
import wandb

     |████████████████████████████████| 1.7 MB 4.2 MB/s 
     |████████████████████████████████| 140 kB 85.1 MB/s 
     |████████████████████████████████| 97 kB 9.5 MB/s 
     |████████████████████████████████| 180 kB 75.3 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 


In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Train with construction equipments

In [ ]:
# YOLO v5s
!python train.py --hyp hyp.finetune.yaml --img 416 --batch 32 --epochs 80 --data data/construction.yaml --cfg models/yolov5s.yaml --weights yolov5s.pt --name yolo5s_construction_human

wandb: Currently logged in as: hyob (use `wandb login --relogin` to force relogin)
train: weights=yolov5s.pt, cfg=models/yolov5s.yaml, data=data/construction.yaml, hyp=hyp.finetune.yaml, epochs=80, batch_size=32, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, name=yolo5s_construction_human, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-139-g8f35436 torch 1.10.0+cu111 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

hyperparameters: lr0=0.0032, lrf=0.12, momentum=0.843, weight_decay=0.00036, warmup_epochs=2.0, warmup_momentum=0.5, warmup_bias_lr=0.05, box=0.0296, cls=0.243, cls_

## Detect
`python detect.py --weights runs/train/yolo5s_construction_human/weights/best.pt --source 'https://youtu.be/MHaZXSneOGQ'`

https://stackoverflow.com/questions/70075378/error-in-python-script-using-python-vlc-and-pafy

## Download weights

In [ ]:
from google.colab import files

In [ ]:
!zip -r yolo5s_construction_human.zip runs/train/yolo5s_construction_human/
files.download('yolo5s_construction_human.zip')

  adding: runs/train/yolo5s_construction_human/ (stored 0%)
  adding: runs/train/yolo5s_construction_human/labels.jpg (deflated 14%)
  adding: runs/train/yolo5s_construction_human/val_batch2_pred.jpg (deflated 9%)
  adding: runs/train/yolo5s_construction_human/R_curve.png (deflated 5%)
  adding: runs/train/yolo5s_construction_human/val_batch0_labels.jpg (deflated 11%)
  adding: runs/train/yolo5s_construction_human/events.out.tfevents.1639282079.b082e473e265.403.0 (deflated 19%)
  adding: runs/train/yolo5s_construction_human/F1_curve.png (deflated 4%)
  adding: runs/train/yolo5s_construction_human/val_batch1_labels.jpg (deflated 11%)
  adding: runs/train/yolo5s_construction_human/opt.yaml (deflated 47%)
  adding: runs/train/yolo5s_construction_human/train_batch1.jpg (deflated 6%)
  adding: runs/train/yolo5s_construction_human/val_batch2_labels.jpg (deflated 9%)
  adding: runs/train/yolo5s_construction_human/val_batch0_pred.jpg (deflated 11%)
  adding: runs/train/yolo5s_construction_huma

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>